MIMIC_Sepsis
=================

# 1 Preparation

To run this document the following requirements must be satisfied:

- Implement the database mimic in **PostgreSQL** and start it. The instruction can be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres). (The name of this environment should be **mimiciv**)
- generate useful abstractions of raw MIMIC-IV data. The instruction be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/concepts_postgres) 



To install all the required libraries, run:

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.6 MB/s eta 0:00:0000:0100:01
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
  Using cached scikit_learn-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


After all the preparation is done, run the following cell to connect to the database.

In [ ]:
import psycopg2
from psycopg2 import sql
import csv
import pandas as pd
import numpy as np
import os
import shutil
import csv
from datetime import timedelta
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor

# implement the username, password and database name
conn = psycopg2.connect(host='', user='', password='', database='mimiciv')

# 2 Extract selected data from the original database 

According to the paper we need to extract the **state space** and **action space** respectively from the mimiciv database. The table **mimic4 itemid.csv** lists all the items required.

***Uncomment the following line if you first time run the code***

In [ ]:
# # uncomment the following line if you first time run the code

# # Read the SQL file

# try:
#     with open('sql_file/select_patients_cohort.sql', 'r') as file0:
#         sql_script_select_patients_cohort = file0.read()
        
#     with open('sql_file/state_from_chartevents.sql', 'r') as file1:
#         sql_script_state = file1.read()

#     with open('sql_file/action_from_inputevents.sql', 'r') as file2:
#         sql_script_action = file2.read()

#     # Execute the SQL script
#     cursor = conn.cursor()
    
#     cursor.execute(sql.SQL(sql_script_select_patients_cohort))
#     cursor.execute(sql.SQL(sql_script_state))
#     cursor.execute(sql.SQL(sql_script_action))

#     conn.commit()
#     cursor.close()
    
# except (Exception, psycopg2.DatabaseError) as error:
#     print("Error executing SQL statement:", error)

Get the number of stay_ids:

In [ ]:
with conn.cursor() as cursor:
    command = "SELECT distinct stay_id FROM mimiciv_derived.sepsis_patients_cohort;"
    cursor.execute(command)   
    result = cursor.fetchall()
    stay_ids= [row[0] for row in result]
    num_stay_ids = len(stay_ids)
    print('Number of stay_ids: ' + str(num_stay_ids))
    cursor.close()

# 3 Data transfor

## 3.1 Data transfer of State Space
We transfer the data of State Space from Postgresql to csv.